In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import cv2
import os
import glob

import functions as fn

import tensorflow as tf
from tensorflow import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger

seed=42

In [2]:
train_data = keras.preprocessing.image_dataset_from_directory(
    '../created_data/train', 
    labels='inferred',
    subset="training",
    validation_split=.2,
    seed=seed,
    shuffle=True)

val_data = keras.preprocessing.image_dataset_from_directory(
    '../created_data/train', 
    labels='inferred',
    subset="validation",
    validation_split=.2,
    seed=seed,
    shuffle=True)

test_data = keras.preprocessing.image_dataset_from_directory(
    '../created_data/test', 
    labels='inferred',
    shuffle=False)

Found 7936 files belonging to 2 classes.
Using 6349 files for training.
Found 7936 files belonging to 2 classes.
Using 1587 files for validation.
Found 1974 files belonging to 2 classes.


In [3]:

#for images, labels in train_data.take(1):
#    for i in range(3):
#        fn.display_image(images[i].numpy().astype('uint8'))

In [4]:
""" plt.figure(figsize=(10, 10))
for images, labels in train_data.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(train_data.class_names[labels[i]])
        plt.axis("off") """

' plt.figure(figsize=(10, 10))\nfor images, labels in train_data.take(1):\n    for i in range(9):\n        ax = plt.subplot(3, 3, i + 1)\n        plt.imshow(images[i].numpy().astype("uint8"))\n        plt.title(train_data.class_names[labels[i]])\n        plt.axis("off") '

In [5]:
def preprocess(image, label):
    #image =  image.astype(np.float32) / 255.
    resized_image = tf.image.resize(image, [512,512])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

In [6]:
train_data = train_data.map(preprocess).prefetch(1)
val_data = val_data.map(preprocess).prefetch(1)
test_data = test_data.map(preprocess).prefetch(1)

In [7]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [10]:
base_model_Xception = keras.applications.xception.Xception(weights = 'imagenet', include_top = False)

model_dir = '../models'
model_uuid = 'model_Xception'

for layer in base_model_Xception.layers:
    layer.trainable = False

avg = keras.layers.GlobalAveragePooling2D()(base_model_Xception.output)
output = keras.layers.Dense(1, activation = 'sigmoid')(avg)
model_Xception = keras.Model(inputs = base_model_Xception.input, outputs = output)

early_stopping = EarlyStopping(monitor='val_loss', verbose=2, patience=0, min_delta=.00250)
model_checkpoint = ModelCheckpoint(f'{model_dir}/{model_uuid}_weights{{epoch:08d}}.h5', verbose = 2, save_best_only=False, period=1)
csv_logger = CSVLogger(f'{model_dir}/{model_uuid}.csv', separator = ',', append = True)

optimizer = keras.optimizers.SGD(learning_rate = 0.2, momentum = 0.9, decay = 0.01)
model_Xception.compile(loss = 'binary_crossentropy', optimizer = 'adam',  metrics = ['accuracy', recall_m, precision_m, f1_m])

results = model_Xception.fit_generator(train_data,
    epochs=4,
    validation_data=val_data,
    callbacks=[early_stopping, model_checkpoint, csv_logger])

Epoch 1/4


C:\Users\nicho\AppData\Local\Temp\ipykernel_1084\3638164968.py:20: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  results = model_Xception.fit_generator(train_data,


50/50 [==============================] - ETA: 0s - loss: 0.6980 - accuracy: 0.5142 - recall_m: 0.7058 - precision_m: 0.5183 - f1_m: 0.5685
Epoch 1: saving model to ../models\model_Xception_weights00000001.h5
50/50 [==============================] - 297s 6s/step - loss: 0.6980 - accuracy: 0.5142 - recall_m: 0.7058 - precision_m: 0.5183 - f1_m: 0.5685 - val_loss: 0.6828 - val_accuracy: 0.5545 - val_recall_m: 0.3257 - val_precision_m: 0.6040 - val_f1_m: 0.4130
Epoch 2/4
50/50 [==============================] - ETA: 0s - loss: 0.6813 - accuracy: 0.5608 - recall_m: 0.5898 - precision_m: 0.5814 - f1_m: 0.5336
Epoch 2: saving model to ../models\model_Xception_weights00000002.h5
50/50 [==============================] - 289s 6s/step - loss: 0.6813 - accuracy: 0.5608 - recall_m: 0.5898 - precision_m: 0.5814 - f1_m: 0.5336 - val_loss: 0.6681 - val_accuracy: 0.6144 - val_recall_m: 0.5995 - val_precision_m: 0.6223 - val_f1_m: 0.6043
Epoch 2: early stopping


In [12]:
model_Xception.evaluate(test_data)

62/62 [==============================] - 178s 3s/step - loss: 0.6878 - accuracy: 0.5355 - recall_m: 0.2531 - precision_m: 0.4966 - f1_m: 0.3325


[0.6878007054328918,
 0.5354610085487366,
 0.25312501192092896,
 0.49660441279411316,
 0.3324719965457916]

In [14]:
base_model_Xception = keras.applications.xception.Xception(weights = 'imagenet', include_top = False)

model_dir = '../models'
model_uuid = 'model_Xception2'

for layer in base_model_Xception.layers:
    layer.trainable = False

avg = keras.layers.GlobalAveragePooling2D()(base_model_Xception.output)
output = keras.layers.Dense(1, activation = 'sigmoid')(avg)
model_Xception = keras.Model(inputs = base_model_Xception.input, outputs = output)

model_Xception.load_weights(f'{model_dir}/model_Xception_weights00000002.h5')

early_stopping = EarlyStopping(monitor='val_loss', verbose=2, patience=3, min_delta=.00250)
model_checkpoint = ModelCheckpoint(f'{model_dir}/{model_uuid}_weights{{epoch:08d}}.h5', verbose = 2, save_best_only=False, period=1)
csv_logger = CSVLogger(f'{model_dir}/{model_uuid}.csv', separator = ',', append = True)

model_Xception.compile(loss = 'binary_crossentropy', optimizer = 'adam',  metrics = ['accuracy', recall_m, precision_m, f1_m])

results = model_Xception.fit_generator(train_data,
    epochs=20,
    validation_data=val_data,
    callbacks=[early_stopping, model_checkpoint, csv_logger])

Epoch 1/20


C:\Users\nicho\AppData\Local\Temp\ipykernel_1084\4262946862.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  results = model_Xception.fit_generator(train_data,


50/50 [==============================] - ETA: 0s - loss: 0.6714 - accuracy: 0.5892 - recall_m: 0.6364 - precision_m: 0.5991 - f1_m: 0.5842
Epoch 1: saving model to ../models\model_Xception2_weights00000001.h5
50/50 [==============================] - 290s 6s/step - loss: 0.6714 - accuracy: 0.5892 - recall_m: 0.6364 - precision_m: 0.5991 - f1_m: 0.5842 - val_loss: 0.6587 - val_accuracy: 0.6270 - val_recall_m: 0.7774 - val_precision_m: 0.6001 - val_f1_m: 0.6706
Epoch 2/20
50/50 [==============================] - ETA: 0s - loss: 0.6632 - accuracy: 0.6062 - recall_m: 0.6909 - precision_m: 0.6164 - f1_m: 0.6229
Epoch 2: saving model to ../models\model_Xception2_weights00000002.h5
50/50 [==============================] - 288s 6s/step - loss: 0.6632 - accuracy: 0.6062 - recall_m: 0.6909 - precision_m: 0.6164 - f1_m: 0.6229 - val_loss: 0.6522 - val_accuracy: 0.6200 - val_recall_m: 0.8439 - val_precision_m: 0.5836 - val_f1_m: 0.6839
Epoch 3/20
50/50 [==============================] - ETA: 0s - l

In [15]:
model_Xception.evaluate(test_data)

62/62 [==============================] - 178s 3s/step - loss: 0.7005 - accuracy: 0.5441 - recall_m: 0.3175 - precision_m: 0.4964 - f1_m: 0.3852


[0.7004950642585754,
 0.5440729260444641,
 0.3174999952316284,
 0.49641579389572144,
 0.38519051671028137]

In [8]:
base_model_Xception = keras.applications.xception.Xception(weights = 'imagenet', include_top = False)

model_dir = '../models'
model_uuid = 'model_Xception3'

for layer in base_model_Xception.layers:
    layer.trainable = True

avg = keras.layers.GlobalAveragePooling2D()(base_model_Xception.output)
output = keras.layers.Dense(1, activation = 'sigmoid')(avg)
model_Xception = keras.Model(inputs = base_model_Xception.input, outputs = output)

model_Xception.load_weights(f'{model_dir}/model_Xception2_weights00000020.h5')

early_stopping = EarlyStopping(monitor='val_loss', verbose=2, patience=3, min_delta=.00250)
model_checkpoint = ModelCheckpoint(f'{model_dir}/{model_uuid}_weights{{epoch:08d}}.h5', verbose = 2, save_best_only=False, period=1)
csv_logger = CSVLogger(f'{model_dir}/{model_uuid}.csv', separator = ',', append = True)

model_Xception.compile(loss = 'binary_crossentropy', optimizer = 'adam',  metrics = ['accuracy', recall_m, precision_m, f1_m])

results = model_Xception.fit_generator(train_data,
    epochs=5,
    validation_data=val_data,
    callbacks=[early_stopping, model_checkpoint, csv_logger])

Epoch 1/5


C:\Users\nicho\AppData\Local\Temp\ipykernel_724\2845532247.py:21: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  results = model_Xception.fit_generator(train_data,


50/50 [==============================] - ETA: 0s - loss: 0.7067 - accuracy: 0.5098 - recall_m: 0.0654 - precision_m: 0.0621 - f1_m: 0.0628 
Epoch 1: saving model to ../models\model_Xception3_weights00000001.h5
50/50 [==============================] - 950s 19s/step - loss: 0.7067 - accuracy: 0.5098 - recall_m: 0.0654 - precision_m: 0.0621 - f1_m: 0.0628 - val_loss: 0.6934 - val_accuracy: 0.4965 - val_recall_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_f1_m: 0.0000e+00
Epoch 2/5
34/50 [===================>..........] - ETA: 3:22 - loss: 0.6937 - accuracy: 0.4917 - recall_m: 0.0000e+00 - precision_m: 0.0000e+00 - f1_m: 0.0000e+00

In [13]:


early_stopping = EarlyStopping(monitor='val_loss', verbose=2, patience=3, min_delta=.00250)
model_checkpoint = ModelCheckpoint(f'{model_dir}/{model_uuid}_weights{{epoch:08d}}.h5', verbose = 2, save_best_only=False, period=1)
csv_logger = CSVLogger(f'{model_dir}/{model_uuid}.csv', separator = ',', append = True)

model_Xception.compile(loss = 'binary_crossentropy', optimizer = 'adam',  metrics = ['accuracy', recall_m, precision_m, f1_m])

results = model_Xception.fit_generator(train_data,
    epochs=20,
    validation_data=val_data,
    callbacks=[early_stopping, model_checkpoint, csv_logger])

Epoch 1/20


C:\Users\nicho\AppData\Local\Temp\ipykernel_1084\362580371.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  results = model_Xception.fit_generator(train_data,



Epoch 1: saving model to ../models\model_Xception2_weights00000001.h5
 1/50 [..............................] - ETA: 4:17 - loss: 0.6846 - accuracy: 0.5625 - recall_m: 0.5263 - precision_m: 0.6667 - f1_m: 0.5882
Epoch 1: saving model to ../models\model_Xception2_weights00000001.h5
 2/50 [>.............................] - ETA: 2:27 - loss: 0.6831 - accuracy: 0.5156 - recall_m: 0.5263 - precision_m: 0.6111 - f1_m: 0.5644
Epoch 1: saving model to ../models\model_Xception2_weights00000001.h5
 3/50 [>.............................] - ETA: 2:25 - loss: 0.6859 - accuracy: 0.5208 - recall_m: 0.6398 - precision_m: 0.5741 - f1_m: 0.5876
Epoch 1: saving model to ../models\model_Xception2_weights00000001.h5
 4/50 [=>............................] - ETA: 2:22 - loss: 0.6811 - accuracy: 0.5391 - recall_m: 0.7298 - precision_m: 0.5645 - f1_m: 0.6151
Epoch 1: saving model to ../models\model_Xception2_weights00000001.h5
 5/50 [==>...........................] - ETA: 2:19 - loss: 0.6795 - accuracy: 0.5562 

KeyboardInterrupt: 

In [ ]:
model.save('../models/model1')

In [9]:
acc = results.history['accuracy']
val_acc = results.history['val_accuracy']

loss = results.history['loss']
val_loss = results.history['val_loss']

recall = results.history['recall_m']
val_recall = results.history['val_recall_m']

precision = results.history['precision_m']
val_precision = results.history['val_precision_m']

f1 = results.history['f1_m']
val_f1 = results.history['val_f1_m']

epochs_range = range(len(results.history['accuracy']))

plt.figure(figsize=(22, 18))
plt.subplot(2, 3, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 3, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.subplot(2, 3, 3)
plt.plot(epochs_range, recall, label='Training Recall')
plt.plot(epochs_range, val_recall, label='Validation Recall')
plt.legend(loc='lower right')
plt.title('Training and Validation Recall')
plt.show()

plt.subplot(2, 3, 4)
plt.plot(epochs_range, precision, label='Training Precision')
plt.plot(epochs_range, val_precision, label='Validation Precision')
plt.legend(loc='lower right')
plt.title('Training and Validation Precision')
plt.show()

plt.subplot(2, 3, 5)
plt.plot(epochs_range, f1, label='Training F1')
plt.plot(epochs_range, val_f1, label='Validation F1')
plt.legend(loc='lower right')
plt.title('Training and Validation F1')
plt.show()

: 

: 